[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/planetlabs/notebooks/blob/master/jupyter-notebooks/api_guides/tile_services/mapping_basemap_tiles_bokeh.ipynb)

# Tile Services Mapping Demo with Bokeh

This notebook demonstrates how to render Planet Basemaps via [XYZ Tile Services](https://docs.planet.com/develop/apis/tiles/#basemap-tile-service).

In this example, you will build an interactive map plots with [Bokeh](https://bokeh.org/) that looks like this one:

![bokeh_plot.png](images/bokeh_plot.png)

Additionally, you will see how to make linked plots so that you can zoom and pan across multiple maps together:

![bokeh_plot_zoomed_in.png](images/bokeh_plot_zoomed_in.png)

## Setup

In [1]:
# ## install dependencies if needed using your preferred package manager
# !pip install bokeh jupyter_bokeh xyzservices

In [ ]:
import os
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show
from xyzservices import TileProvider

In [ ]:
# setup Planet API key
pl_api_key = os.getenv('PL_API_KEY')
if not pl_api_key:
    import getpass
    pl_api_key = getpass.getpass('Planet API Key: ')

In [5]:
# You may need to adjust this to a location within your own Area of Access
lat, lon = 37.7749, -122.4194

In [6]:
output_notebook()

Loading BokehJS ...

## Make the Maps

In [ ]:
basemap_names = ['global_monthly_2024_11_mosaic', 'global_monthly_2025_02_mosaic', 'global_monthly_2025_05_mosaic', 'global_monthly_2025_08_mosaic']

# Create a list to store the figure objects
plots = []

# Create each map figure
for basemap in basemap_names:
    provider = TileProvider(
        name=basemap,
        url=f'https://tiles.planet.com/basemaps/v1/planet-tiles/{basemap}/gmap/{{z}}/{{x}}/{{y}}.png?api_key={pl_api_key}',
        attribution='Planet Labs PBC'
    )
    
    p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
               x_axis_type="mercator", y_axis_type="mercator",
               width=400, height=400,  # Set dimensions for each plot
               active_scroll="wheel_zoom",
               active_drag="pan",
               title=basemap.replace('global_monthly_', '').replace('_mosaic', ''))
    
    p.add_tile(provider)
    plots.append(p)

# Arrange plots in a 1x4 grid
grid = gridplot([plots], 
                sizing_mode="scale_width")

# Show the combined plot
show(grid)

You should now see 4 separately rendered map figures in a grid with independent control. Try zooming and panning.

In [ ]:
# Create a list to store the figure objects
plots = []

# Create the first plot to use as a reference for linking
ref_plot = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
                 x_axis_type="mercator", y_axis_type="mercator",
                 width=400, height=400,
                 active_scroll="wheel_zoom",
                 active_drag="pan",
                 title=basemap_names[0].replace('global_monthly_', '').replace('_mosaic', ''))

provider = TileProvider(
    name=basemap_names[0],
    url=f'https://tiles.planet.com/basemaps/v1/planet-tiles/{basemap_names[0]}/gmap/{{z}}/{{x}}/{{y}}.png?api_key={pl_api_key}',
    attribution='Planet Labs PBC'
)
ref_plot.add_tile(provider)
plots.append(ref_plot)

# Create the rest of the plots linked to the first one
for basemap in basemap_names[1:]:
    provider = TileProvider(
        name=basemap,
        url=f'https://tiles.planet.com/basemaps/v1/planet-tiles/{basemap}/gmap/{{z}}/{{x}}/{{y}}.png?api_key={pl_api_key}',
        attribution='Planet Labs PBC'
    )
    
    # Create new plot with ranges linked to the reference plot
    p = figure(x_range=ref_plot.x_range, y_range=ref_plot.y_range,
               x_axis_type="mercator", y_axis_type="mercator",
               width=400, height=400,
               active_scroll="wheel_zoom",
               active_drag="pan",
               title=basemap.replace('global_monthly_', '').replace('_mosaic', ''))
    
    p.add_tile(provider)
    plots.append(p)

# Arrange plots in a 1x4 grid
grid = gridplot([plots], 
                sizing_mode="scale_width")

# Show the combined plot
show(grid)

Now try zooming and panning and the separate maps in the grid should stay linked.